In [1]:
import pandas as pd
import numpy as np

In [4]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [6]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    flags = np.isin(np.array(bought_list), np.array(recommended_list)[:k])
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [7]:
hit_rate(recommended_list, bought_list)

1

In [8]:
hit_rate_at_k(recommended_list, bought_list, k=1)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [60, 400,  40, 40 , 90]  
flags = [1, 0,  0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [9]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assert len(bought_list) > len(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
 
    # создаем флаги тех товаров, которые мы купили
    flags = np.isin(recommended_list[:k], bought_list)    

    # возвращаем скалярное произведение, которое нам даст сумму рекомендованных до k покупок, 
    # деленное на сумму всех рекомендованных товаров до k.
    return flags@prices_recommended[:k] / prices_recommended[:k].sum()

In [10]:
precision(recommended_list, bought_list)

0.2

In [11]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [12]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [13]:
prices_recommended = [10, 20, 30, 40, 50, 60, 70, 80, 90, 10]

In [14]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=10)

0.10869565217391304

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [15]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
 
    # создаем флаги тех товаров, которые мы купили
    flags = np.isin(recommended_list[:k], bought_list)

    # возвращаем скалярное произведение, которое нам даст сумму рекомендованных до k покупок, 
    # деленное на сумму всех купленных товаров.
    return flags@prices_recommended[:k] / prices_bought.sum()

In [16]:
recall(recommended_list, bought_list)

0.5

In [17]:
recall_at_k(recommended_list, bought_list, k=3)

0.25

In [18]:
prices_bought = [30, 60, 10, 40]

In [19]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.35714285714285715

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k из лекции:
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

# ВАЖНО

Я изменил функцию подсчета AP@K, на ту, которая доступна по ссылке [здесь](https://habr.com/ru/company/econtenta/blog/303458/)

Я вижу, что в ноутбуке написано, что формулы в статье могут отличаться от формул в лекций, но меня это смутило и запутало, поэтому я просто выбрал формулу с Хабра. Еще одной причиной выбора именно этой формулы, стала [эта](https://medium.com/@misty.mok/how-mean-average-precision-at-k-map-k-can-be-more-useful-than-other-evaluation-metrics-6881e0ee21a9) статья с хорошими объяснениями и примерами.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{K} \sum_{k=1}^{K}{r^{true}(\pi^{-1}(k)) * precision@k}$$

In [20]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    precision_sum = 0
    
    for i, flag in enumerate(flags):
        if flag:
            precision_sum += precision_at_k(recommended_list, bought_list, k=i+1)

    return precision_sum/len(bought_list)

In [21]:
ap_k(recommended_list, bought_list, k=5)

0.375

Закомментированный вариант из лекции:

In [22]:
# def ap_k(recommended_list, bought_list, k=5):
    
#     bought_list = np.array(bought_list)
#     recommended_list = np.array(recommended_list)
    
#     flags = np.isin(recommended_list, bought_list)
#     print(flags)
    
#     if sum(flags) == 0:
#         return 0
    
#     sum_ = 0
#     for i in range(0, k-1):
#         print('i: ',i)
#         if flags[i] == True:
#             print('flags[i]: ', flags[i])
#             p_k = precision_at_k(recommended_list, bought_list, k=i+1)
#             print('p_k: ',p_k)
#             sum_ += p_k
#             print('sum: ',sum_)
            
#     result = sum_ / sum(flags)
    
#     return result

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [23]:
recommended_list_mapk = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],] #id товаров
bought_list_mapk = [[521, 32, 143, 991], [146, 29]]

In [24]:
def map_k(recommended_list, bought_list, k=5, u=1):
    
    return np.mean([ap_k(p,a,k) for a,p in zip(bought_list, recommended_list)])

In [25]:
map_k(recommended_list_mapk, bought_list_mapk)

0.4375

Проверим правильность функции map_k

Посчитаем ap_k для каждого юзера и посмотрим их среднее:

In [26]:
ap_k_user1 = ap_k(recommended_list_mapk[0], bought_list_mapk[0], k=5)
ap_k_user2 = ap_k(recommended_list_mapk[1], bought_list_mapk[1], k=5)
print('User 1 ap_k:',ap_k_user1)
print('User 2 ap_k:',ap_k_user2)
print('Mean ap_k:', (ap_k_user1+ap_k_user2)/2)

User 1 ap_k: 0.375
User 2 ap_k: 0.5
Mean ap_k: 0.4375


### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


https://pytorch.org/docs/stable/index.html

In [27]:
import torch
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [28]:
ys_true

tensor([2, 0, 0, 2, 3])

In [29]:
ys_pred

tensor([0.3938, 0.3867, 0.0762, 0.3713, 0.1828])

In [30]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)

In [31]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme) # считаем гейн
        ret += gain/np.log2(idx+1) # делим полученный гейн на логарифм от индекса + 1
    return ret

In [32]:
dcg(ys_true, ys_pred, gain_scheme='const')

4.29202967422018

In [33]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    pred_dcg = dcg(ys_true, ys_pred, gain_scheme)
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme) # идеальный dcg это когда y_pred = y_true
    ndcg = pred_dcg / ideal_dcg
    return ndcg

In [34]:
ndcg(ys_true, ys_pred)

0.8156868628654561

Сверим со встроенными в sklearn dcg и ndcg скорами:

In [35]:
from sklearn.metrics import ndcg_score, dcg_score

In [36]:
dcg_score([list(ys_true)], [list(ys_pred)])

4.292029674220179

In [37]:
ndcg_score([list(ys_true)], [list(ys_pred)])

0.815686862865456

Все сходится!

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

[MRR](https://drive.google.com/file/d/1uMFKqWULFmehS0pJtvXKRygjYxQOPise/view?usp=sharing)

In [38]:
recommended_list

[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]

In [39]:
bought_list

[521, 32, 143, 991]

In [40]:
recommended_list.index(991)

3

In [41]:
def reciprocal_rank(recommended_list, bought_list):
    result = 0
    for purchase in bought_list:
        if purchase in recommended_list:
            result += 1/(recommended_list.index(purchase)+1)
    return result/len(bought_list)

In [42]:
reciprocal_rank(recommended_list, bought_list)

0.3125